## Financial Product Recommendation Engine

This Notebook depicts steps in creating and saving a model in IBM cloud using Watson Studio

Author : Mukesh R, Sam Prasanna R

### Step 1

The Below code blocks helps to create connections required to save the model and connect to WML

In [59]:
api_key = 'teuonH42NK9fs-h2kbKROGBHhiirxc6tFhyXQ7eIOcgg'
location = 'us-south'

In [60]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [3]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.4MB 9.1MB/s eta 0:00:01
  Found existing installation: ibm-watson-machine-learning 1.0.8
    Uninstalling ibm-watson-machine-learning-1.0.8:
      Successfully uninstalled ibm-watson-machine-learning-1.0.8


In [61]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [64]:
space_id = 'f6f6501f-c8e1-4c4f-9c65-59ecafdcc386'

In [65]:
client.spaces.list(limit=10)

------------------------------------  -----------------------  ------------------------
ID                                    NAME                     CREATED
f6f6501f-c8e1-4c4f-9c65-59ecafdcc386  lticiti-hackathon-space  2020-09-05T19:00:04.389Z
------------------------------------  -----------------------  ------------------------


In [66]:
client.set.default_space(space_id)

'SUCCESS'

### Step2 

Connect to the Dataset uploaded in Projects in Watson Studio

In [26]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6a94a2d51103455f82f9a755c24f1ed0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='yVud9NTK3llNtltIXHHRTs-CF8AfkLAbvIg5bfCio--n',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6a94a2d51103455f82f9a755c24f1ed0.get_object(Bucket='lticitihackathonpro-donotdelete-pr-lpkddav7n9w3qp',Key='train_loan.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_2 = pd.read_csv(body)
df_data_2.head()


,age,job,marital status,education,credit default?,housing loan?,Vehicle loan,y
0,30,unemployed,married,primary,no,no,no,no
1,33,services,married,secondary,no,yes,yes,no
2,35,management,single,tertiary,no,yes,no,no
3,30,management,married,tertiary,no,yes,yes,no
4,59,blue-collar,married,secondary,no,yes,no,no


In [27]:
obj_df = df_data_2.copy()
obj_df.head()

,age,job,marital status,education,credit default?,housing loan?,Vehicle loan,y
0,30,unemployed,married,primary,no,no,no,no
1,33,services,married,secondary,no,yes,yes,no
2,35,management,single,tertiary,no,yes,no,no
3,30,management,married,tertiary,no,yes,yes,no
4,59,blue-collar,married,secondary,no,yes,no,no


### Step 3 

Check for nulls in the records and Replace them with Mean/Mode/Median or Default Category based on the type of data

Since the ML algorithms doesnt perform well with Null in data

In [28]:
obj_df[obj_df.isnull().any(axis=1)]

,age,job,marital status,education,credit default?,housing loan?,Vehicle loan,y
79,40,NaN,married,secondary,no,yes,no,no
97,30,admin.,single,NaN,no,no,no,no
108,56,unemployed,NaN,primary,no,no,no,yes
109,27,housemaid,married,primary,no,NaN,NaN,no
168,48,technician,married,tertiary,NaN,no,no,no
1000,38,admin.,married,secondary,no,yes,NaN,no
1004,42,NaN,single,secondary,no,yes,yes,yes
1016,33,services,married,secondary,no,NaN,no,no


In [29]:

obj_df["job"] = obj_df["job"].astype('category')
obj_df.dtypes

obj_df["job_cat"] = obj_df["job"].cat.codes
obj_df.head()

obj_df["marital status "] = obj_df["marital status "].astype('category')
obj_df.dtypes

obj_df["marital status_cat"] = obj_df["marital status "].cat.codes
obj_df.head()


obj_df["education"] = obj_df["education"].astype('category')
obj_df.dtypes

obj_df["education_cat"] = obj_df["education"].cat.codes
obj_df.head()


obj_df["credit default?"] = obj_df["credit default?"].astype('category')
obj_df.dtypes

obj_df["credit default_cat"] = obj_df["credit default?"].cat.codes
obj_df.head()


obj_df["housing loan?"] = obj_df["housing loan?"].astype('category')
obj_df.dtypes

obj_df["housing loan_cat"] = obj_df["housing loan?"].cat.codes
obj_df.head()

obj_df["Vehicle loan"] = obj_df["Vehicle loan"].astype('category')
obj_df.dtypes

obj_df["Vehicle loan_cat"] = obj_df["Vehicle loan"].cat.codes
obj_df.head()


obj_df["y"] = obj_df["y"].astype('category')
obj_df.dtypes

obj_df["y_cat"] = obj_df["y"].cat.codes
obj_df.head()#Replace the NAN's with Mode, Since it was given "For Categorical Value replace it with Mode"
obj_df['job'].fillna(obj_df['job'].mode()[0], inplace=True)

obj_df['marital status '].fillna(obj_df['marital status '].mode()[0], inplace=True)
obj_df['education'].fillna(obj_df['education'].mode()[0], inplace=True)
obj_df['credit default?'].fillna(obj_df['credit default?'].mode()[0], inplace=True)
obj_df['housing loan?'].fillna(obj_df['housing loan?'].mode()[0], inplace=True)
obj_df['Vehicle loan'].fillna(obj_df['Vehicle loan'].mode()[0], inplace=True)



### Step 4

Check for imbalance target variable and either do Upsampling / Downsampling depending on the dataset

In [30]:
obj_df['y'].value_counts()

no     897
yes    124
Name: y, dtype: int64

In [31]:
#Steps to balance the class variables
# Separate majority and minority classes
df_majority = obj_df[obj_df.y=='no']
df_minority = obj_df[obj_df.y=='yes']

In [32]:
from sklearn.utils import resample
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=897,    # to match majority class
                                 random_state=123) # reproducible results

In [33]:
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
df_upsampled.y_cat.value_counts()

1    897
0    897
Name: y_cat, dtype: int64

In [34]:
df_upsampled.head()

,age,job,marital status,education,credit default?,housing loan?,Vehicle loan,y,job_cat,marital status_cat,education_cat,credit default_cat,housing loan_cat,Vehicle loan_cat,y_cat
0,30,unemployed,married,primary,no,no,no,no,10,1,0,0,0,0,0
1,33,services,married,secondary,no,yes,yes,no,7,1,1,0,2,1,0
2,35,management,single,tertiary,no,yes,no,no,4,2,2,0,2,0,0
3,30,management,married,tertiary,no,yes,yes,no,4,1,2,0,2,1,0
4,59,blue-collar,married,secondary,no,yes,no,no,1,1,1,0,2,0,0


### Step 5

Since this is POC to show how classification can be helpful in acheiving usecases and sample dataset has very less columns we did not do feature engineering to either remove column which doesnot have impact on target varaibles (eg : Customer ID) using PCA and other methods

The below step invloves getting the required input and output dataframes

Also, Splitting the data into 70 - 30 (Train - Test Split) to Rate the model performance using Confusion Matrix

In this we have used accuracy has the measure to check performance of different algorithms

In [35]:
Y = df_upsampled.iloc[:,14]  
X = df_upsampled.iloc[:,[0,8,9,10,11,12,13]]

In [36]:
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

### Step 6

Define Model, train and evaluate them

In [37]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy")

# Train Decision Tree Classifer
clf = clf.fit(X_train,Y_train)

#Predict the response for test dataset
Y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.9313543599257885


In [38]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100,
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train,Y_train)

#Predict the response for test dataset
Y_pred = model.predict(X_test)

print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

Accuracy: 0.922077922077922


In [39]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# Train the model using the training sets
model.fit(X_train,Y_train)

#Predict Output
predicted= model.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.5862708719851577


In [40]:
from sklearn.naive_bayes import BernoulliNB
model = BernoulliNB()

# Train the model using the training sets
model.fit(X_train,Y_train)

#Predict Output
predicted= model.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.5677179962894249


In [41]:
from sklearn import svm

svm = svm.SVC()

# Train the model using the training sets
svm.fit(X_train,Y_train)

#Predict Output
predicted= svm.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.7903525046382189


In [42]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(alpha=0.1, hidden_layer_sizes=(10,))

# Train the model using the training sets
MLP.fit(X_train,Y_train)

#Predict Output
predicted= MLP.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.5844155844155844


In [43]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

#ada = AdaBoostClassifier(n_estimators=100)

ada = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.01, random_state=3).fit(X_train, Y_train)

#Predict Output
predicted= ada.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.7346938775510204


In [44]:
from xgboost import XGBClassifier

# fit model no training data
model = XGBClassifier(max_depth = 10, subsample=0.8,eta = 0.1)
model.fit(X_train, Y_train)

predicted= model.predict(X_test) 

print("Accuracy:",metrics.accuracy_score(Y_test, predicted))

Accuracy: 0.9128014842300557


### Step 7 

Save the model to IBM deployment space to deploy it as API and Make Prediction in Single/Batch

In [45]:
sofware_spec_uid = client.software_specifications.get_id_by_name("scikit-learn_0.20-py3.6")

In [67]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.20',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=clf,
    meta_props=metadata,
    training_data=X_train,
    training_target=Y_train)

In [68]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "label_column": "y_cat",
    "software_spec": {
      "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
      "name": "scikit-learn_0.20-py3.6"
    },
    "training_data_references": [
      {
        "connection": {
          "access_key_id": "not_applicable",
          "endpoint_url": "not_applicable",
          "secret_access_key": "not_applicable"
        },
        "id": "1",
        "location": {},
        "schema": {
          "fields": [
            {
              "name": "age",
              "type": "int64"
            },
            {
              "name": "job_cat",
              "type": "int8"
            },
            {
              "name": "marital status_cat",
              "type": "int8"
            },
            {
              "name": "education_cat",
              "type": "int8"
            },
            {
              "name": "credit default_cat",
              "type": "int8"
            },
            {
              "name": "housing loan_cat"

In [69]:
models_details = client.repository.list_models()

------------------------------------  ----------------------------------------------------------------  ------------------------  -----------------
ID                                    NAME                                                              CREATED                   TYPE
d2e66f5d-a060-445d-afa8-b0f7351d2fa4  Scikit model                                                      2020-09-08T08:32:10.002Z  scikit-learn_0.20
f603c32a-cab7-468c-9e4b-1697ddca1ad9  Trade Settlement Regression model                                 2020-09-06T16:30:25.002Z  scikit-learn_0.20
9ca56be4-7517-4c8c-ac02-3625c3d8c101  TicketCategoryClassifier                                          2020-09-06T12:09:14.002Z  scikit-learn_0.22
902a0eb8-3419-4c42-92d6-4a8dac0103f7  Scikit model                                                      2020-09-06T10:42:41.002Z  scikit-learn_0.20
eaf64ba8-8d14-4639-8aa4-033e3a3f88e1  TicketCategoryClassifier                                          2020-09-06T10:39:07.0

In [70]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Personal Loan",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'd2e66f5d-a060-445d-afa8-b0f7351d2fa4' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='85280b66-0f77-40a5-b420-a2ac53ea18d0'
------------------------------------------------------------------------------------------------




In [71]:
deployment_uid = client.deployments.get_uid(created_deployment)

In [72]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/85280b66-0f77-40a5-b420-a2ac53ea18d0/predictions


In [73]:
client.deployments.list()

------------------------------------  -----------------------------------  -----  ------------------------
GUID                                  NAME                                 STATE  CREATED
85280b66-0f77-40a5-b420-a2ac53ea18d0  Personal Loan                        ready  2020-09-08T08:32:14.187Z
d1c52463-788b-4cf6-853e-787e0e09004e  Deployment of regression model       ready  2020-09-06T16:30:53.615Z
fd67aabb-a3c0-41a9-92c7-8591881a0c5f  TicketCategoryClassifier Deployment  ready  2020-09-06T12:09:18.078Z
7faa31c0-7fa2-41d2-b0da-b93a685c3d95  TicketCategoryClassifier Deployment  ready  2020-09-06T10:44:02.347Z
------------------------------------  -----------------------------------  -----  ------------------------
